## Heatmap

In [ ]:
pip install psycopg2-binary

Or ...


In [ ]:
pip install pandas

In [ ]:
pip install psycopg2

In [ ]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(database="nutanix", 
                        user="postgres", 
                        host='172.25.221.34',
                        password="Senna",
                        port=1433)

In [ ]:
import pandas as pd

try:
    # Create a cursor
    cur = conn.cursor()

    # Execute a query to count the total number of rows in the ssd_perf_data table
    cur.execute("SELECT COUNT(*) FROM ssd_perf_data;")
    total_rows = cur.fetchone()[0]  # Fetch the count result

    # Print the total number of rows
    print(f"Total number of rows in ssd_perf_data: {total_rows}")

    # Execute a query to select the first 10 rows from the ssd_perf_data table
    cur.execute("SELECT * FROM ssd_perf_data LIMIT 200000;")
    rows = cur.fetchall()  # Fetch all results

    # Print the number of rows returned by the LIMIT query
    # print(f"Number of rows returned: {len(rows)}")

    # Print the contents of the table
    # for row in rows:
    #     print(row)  # Print each row
        
    # Store Each Values into a DF 
    # Get column names from cursor description
    column_names = [desc[0] for desc in cur.description]

    # Pass the result into a pandas DataFrame
    df = pd.DataFrame(rows, columns=column_names)
    
    # Print the Data Frame 
    print(df)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()


## K-Means Clustering (Messing Around w/ GPT)

In [ ]:
pip install scikit-learn


In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

### Example of 2D Plot Clustering (REALLY BAD)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming `df` is your DataFrame with SSD performance data
# Select numeric features and normalize
features = df.select_dtypes(include=['float64', 'int64']).dropna()
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Apply t-SNE for dimensionality reduction to 2 components
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
features_tsne = tsne.fit_transform(features_normalized)

# Assuming you've already clustered data and added a 'tier' column
k = 4  # Assuming you have k=4 clusters from K-Means
df['tier'] = KMeans(n_clusters=k, random_state=42).fit_predict(features_normalized)

# Plot the t-SNE clusters
plt.scatter(features_tsne[:, 0], features_tsne[:, 1], c=df['tier'], cmap='viridis')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('SSD Performance Tiers (t-SNE Visualization)')
plt.colorbar(label='Cluster Label')
plt.show()


### Example of Heat Map with no Categorical Data (KINDA BAD)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Assume `df` is your DataFrame containing the SSD performance data
# Select numeric features and normalize them
numeric_df = df.select_dtypes(include=['float64', 'int64']).dropna()
scaler = StandardScaler()
features_normalized = scaler.fit_transform(numeric_df)

# Lower blocksize Dominance 
df[numeric_df] = df[numeric_df].apply(lambda x: np.log1p(x))

# Apply K-Means clustering
k = 10  # Number of clusters/tier levels
kmeans = KMeans(n_clusters=k, random_state=42)
df['tier'] = kmeans.fit_predict(features_normalized)

# Select only numeric columns for aggregation
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
cluster_summary = df.groupby('tier')[numeric_columns].mean()

# Create a heatmap of the average metrics per cluster
plt.figure(figsize=(10, 6))
sns.heatmap(cluster_summary.T, cmap='viridis', annot=True, fmt=".2f")
plt.title('Cluster Feature Averages Heatmap')
plt.xlabel('Cluster (Tier)')
plt.ylabel('Feature')
plt.show()
